In [28]:
from podcast_ad_skipper.google_cloud import *
from podcast_ad_skipper.params import *
from podcast_ad_skipper.data_preparation import *
import pandas as pd
import json
from podcast_ad_skipper.main import get_processed_training_data

In [29]:
storage_client = auth_gc_storage()
columns = '["spectrogram", "labels"]'

Running in local environment.
Authenticated successfully with GCS! ✅


In [30]:
bucket_name = BUCKET_NAME
prefixes=GCP_PREFIXES[0]
chunk_size=5


In [31]:
file_list = retrieve_files_in_folder(storage_client, bucket_name, prefixes)

In [32]:
chunks_list = make_chunks(file_list, chunk_size)

In [33]:
# %%time

bq_client = auth_gc_bigquery()

for chunk in chunks_list[:200]:

    features = get_features_model(chunk)

    table_id = f"{GCP_PROJECT_ID}.{BQ_DATASET}.test10"

    rows_to_insert = [
        {
            "spectrogram": json.dumps(features[0][i].tolist()),
            "labels": features[1][i],
            "seconds": features[2][i],
            "duration": features[3][i],
            "podcast_names": features[4][i],
        } for i in range(len(features[0]))
    ]
    insert_data_to_bq(rows_to_insert, bq_client, table_id, chunk_size)
    # Time out error again, with both [:1] and [:10]
    # Move spectrogram code to model pipeline

Running in local environment.
Authenticated successfully with BigQuery! ✅
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added

In [38]:
table_id = f"{GCP_PROJECT_ID}.{BQ_DATASET}.test10"

In [39]:
bq_client = auth_gc_bigquery()

Running in local environment.
Authenticated successfully with BigQuery! ✅


In [41]:
output= get_output_query_bigquery(bq_client, table_id, limit=None, columns="*")


Query returned 1005 results


In [6]:
type(output)

google.cloud.bigquery.table.RowIterator

In [42]:
output_processed = get_bq_processed_data(output)

In [13]:
output_processed

([array([[-47.4757309 , -53.66692352, -64.69184875, ..., -67.02127838,
          -65.63458252, -61.32347488],
         [-44.6698761 , -50.49269485, -58.24084091, ..., -62.07104492,
          -66.32159424, -62.47253036],
         [-40.27104187, -44.763134  , -53.42609024, ..., -56.8628273 ,
          -60.5264473 , -58.4808197 ],
         ...,
         [-62.51431656, -64.42776489, -69.13052368, ..., -70.65504456,
          -80.        , -80.        ],
         [-61.25905609, -60.63599777, -63.40979385, ..., -70.91630554,
          -80.        , -80.        ],
         [-63.11557007, -64.24725342, -67.78479004, ..., -74.06596375,
          -80.        , -80.        ]]),
  array([[-68.14377594, -67.77185059, -64.17067719, ..., -64.57652283,
          -62.63965607, -59.55200958],
         [-71.99550629, -71.66521454, -78.18234253, ..., -65.41227722,
          -65.47524261, -61.17208099],
         [-76.94216919, -71.9666214 , -68.07937622, ..., -58.81544495,
          -60.77533722, -60.91940

In [43]:
output_processed[0][2].shape

(128, 216)

In [44]:
nans_count = 0
spec_column = output_processed[0]

def check_for_nans(spec_column, nans_count):
    for idx, array in enumerate(spec_column):
        if np.isnan(array).any():
                # print(f"NaNs found in array at index {idx}")
            nans_count += 1
    print(nans_count)

In [45]:
check_for_nans(spec_column, nans_count)

0


In [46]:
# No nans in 1000 rows